# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

!pip install xgboost 

In [54]:
import numpy as np
from sklearn.datasets import load_boston
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
data = load_boston()
X = data['data']
y = data['target']
(X_train,X_test,y_train,y_test) = train_test_split(X,y,test_size=0.25,shuffle=False, random_state = 42)

In [120]:
print(data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [5]:
def function_grad(y,a):
    return -(a-y)

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

**Мой вариант**

In [6]:
base_algorithm = []
coeffs = []

pred = np.zeros((50,y_train.shape[0]))
grad = y_train

for i in range(50):
    reg = DecisionTreeRegressor(max_depth=5,random_state=42)
    reg.fit(X_train,grad)    
    base_algorithm.append( reg )
    coeffs.append(.9)
    pred[i,:] = reg.predict(X_train)*0.9    
    grad = y_train - pred.sum(axis=0)    

In [7]:
def pred_y_fun(X):
    pred = np.zeros(X.shape[0])
    for trees, coef in zip(base_algorithm,coeffs):
        pred += trees.predict(X)*coef
    return pred

y_test_pred = pred_y_fun(X_test)
rmse = mean_squared_error(y_test,y_test_pred,squared=False)
#rmse = (y_test-y_test_pred).std() #Это неправильно, т.к. это разные характеристики
print(rmse)       
 

5.455565103009402


In [80]:
#их функцией воспользуюсь для сравнения
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithm, coeffs)]) for x in X]

y_test_pred = gbm_predict(X_test)
rmse = mean_squared_error(y_test,y_test_pred)**.5
print(rmse)
with open('c:/MatkivskiyV/study/2course/week4/2_t1.txt','w') as f:
    f.write(str(rmse))

5.455565103009402


А это я с форума скопировал, там немного иначе сделано

In [38]:
base_algorithm = []
coeffs = []
s = y_train
for i in range(50):
    clf = DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X=X_train, y=s)
    base_algorithm.append(clf)
    coeffs.append(0.9)
    s = y_train - gbm_predict(X_train)
    
error = mean_squared_error(y_test, gbm_predict(X_test))
rmse = error**.5
print(gbm_predict(X_test)[:5])
print(y_test[:5])
print('\n',rmse)

5.455565103009402


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [8]:
base_algorithm = []
coeffs = []

pred = np.zeros((50,y_train.shape[0]))
grad = y_train

for i in range(50):
    reg = DecisionTreeRegressor(max_depth=5,random_state=42)
    reg.fit(X_train,grad)    
    base_algorithm.append( reg )
    coeffs.append(0.9/(1.0+i))
    pred[i,:] = reg.predict(X_train)*0.9/(1.0+i)    
    grad = y_train - pred.sum(axis=0)
    
def pred_y_fun(X):
    pred = np.zeros(X.shape[0])
    for trees, coef in zip(base_algorithm,coeffs):
        pred += trees.predict(X)*coef
    return pred

y_test_pred = pred_y_fun(X_test)

rmse = mean_squared_error(y_test,y_test_pred,squared=False)
#rmse = (y_test-y_test_pred).std() #Это неправильно, т.к. это разные характеристики
print(rmse)
with open('c:/MatkivskiyV/study/2course/week4/2_t2.txt','w') as f:
    f.write(str(rmse))

4.812550945781193


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

Правильные ответы: 2 4

In [52]:
#n_trees = [10,50,100,200,500,1000,1500]
nt = 10
depth = [2,5,15,30,50,100,300,500,1000]
#scoring_n_trees = [] #np.zeros((3,len(n_trees)))
scoring_n_depths = []

for dp in depth:
    reg = xgb.XGBRegressor(max_depth=dp,n_estimators=nt)
    reg.fit(X_train,y_train)
    y_test_pred = reg.predict(X_test)
    scoring_n_depths.append( mean_squared_error(y_test,y_test_pred,squared=False) )

In [56]:
nt = 10
depth = [2,5,15,30,50,100,300,500,1000]
#scoring_n_trees = [] #np.zeros((3,len(n_trees)))
scoring_n_depths = []
scoring_n_depths

for dp in depth:
    reg = GradientBoostingRegressor(max_depth=dp,n_estimators=nt)
    reg.fit(X_train,y_train)
    y_test_pred = reg.predict(X_test)
    scoring_n_depths.append( mean_squared_error(y_test,y_test_pred,squared=False) )

In [44]:
scoring_n_trees

[4.6686363233333115,
 4.999017672475605,
 5.044208566778392,
 5.04541708765665,
 5.04554895604797,
 5.04554895604797,
 5.04554895604797]

In [57]:
scoring_n_depths

[8.288236557110222,
 6.679492863294032,
 7.468475182520959,
 7.2813795267904275,
 7.152780245409837,
 7.420696614749126,
 7.333103202577545,
 7.2017395906684145,
 7.265549289529715]

In [59]:
with open('c:/MatkivskiyV/study/2course/week4/2_t4.txt','w') as f:
    f.write('2 3')

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [62]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train,y_train)
y_test_pred = reg.predict(X_test)
rmse = mean_squared_error(y_test,y_test_pred,squared=False)
print(rmse)
with open('c:/MatkivskiyV/study/2course/week4/2_t5.txt','w') as f:
    f.write(str(rmse))

8.254979753549161
